In [ ]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display

from duet.process import *
from duet.constants import *

import os

FIGURES = "figures"
if not os.path.exists(FIGURES):
    os.mkdir(FIGURES)

In [ ]:
name = "method_timeline"

now = datetime(year=2022, month=7, day=1, hour=8, minute=20, second=0)
version = "Software version"
method = "method"
df = pd.DataFrame(
    [
        {method: "sequential", version: "A", RF.iteration: 0, RF.start: 0, RF.end: 1},
        {method: "sequential", version: "A", RF.iteration: 1, RF.start: 1, RF.end: 2},
        {method: "sequential", version: "B", RF.iteration: 0, RF.start: 2, RF.end: 4},
        {method: "sequential", version: "B", RF.iteration: 1, RF.start: 4, RF.end: 5},
        {
            method: "synchronous duet",
            version: "A",
            RF.iteration: 0,
            RF.start: 0,
            RF.end: 1,
        },
        {
            method: "synchronous duet",
            version: "B",
            RF.iteration: 0,
            RF.start: 0,
            RF.end: 2,
        },
        {
            method: "synchronous duet",
            version: "A",
            RF.iteration: 1,
            RF.start: 2,
            RF.end: 3,
        },
        {
            method: "synchronous duet",
            version: "B",
            RF.iteration: 1,
            RF.start: 2,
            RF.end: 3,
        },
        {
            method: "asynchronous duet",
            version: "A",
            RF.iteration: 0,
            RF.start: 0,
            RF.end: 1,
        },
        {
            method: "asynchronous duet",
            version: "B",
            RF.iteration: 0,
            RF.start: 0,
            RF.end: 2,
        },
        {
            method: "asynchronous duet",
            version: "A",
            RF.iteration: 1,
            RF.start: 1,
            RF.end: 2,
        },
        {
            method: "asynchronous duet",
            version: "B",
            RF.iteration: 1,
            RF.start: 2,
            RF.end: 3,
        },
    ]
)

df[RF.start] = now + df[RF.start].apply(lambda x: timedelta(minutes=x))
df[RF.end] = now + df[RF.end].apply(lambda x: timedelta(minutes=x))
df[RF.iteration] = df[RF.iteration].astype(str)
fig = px.timeline(
    df, x_start=RF.start, x_end=RF.end, y=version, facet_col=method, color=RF.iteration
)
fig.update_layout(height=300)
fig.update_yaxes(categoryorder="category descending")
fig.write_image(f"{FIGURES}/{name}.pdf")
fig.show()

In [ ]:
name = "overlap_timeline"

now = datetime(year=2022, month=7, day=1, hour=8, minute=20, second=0)
version = "Software version"
overlap = "overlap"
phase = "phase"
df = pd.DataFrame(
    [
        {overlap: "partial", version: "A", RF.iteration: "i", RF.start: 0, RF.end: 1},
        {
            overlap: "partial",
            version: "A",
            RF.iteration: "i+1",
            RF.start: 1.5,
            RF.end: 2.5,
        },
        {overlap: "partial", version: "A", RF.iteration: "i+2", RF.start: 3, RF.end: 4},
        {
            overlap: "partial",
            version: "B",
            RF.iteration: "j",
            RF.start: 0.8,
            RF.end: 1.8,
        },
        {
            overlap: "partial",
            version: "B",
            RF.iteration: "j+1",
            RF.start: 2.3,
            RF.end: 3.3,
        },
        {
            overlap: "partial",
            version: "B",
            RF.iteration: "j+2",
            RF.start: 3.8,
            RF.end: 4.8,
        },
        {overlap: "worst", version: "A", RF.iteration: "i", RF.start: 0, RF.end: 1},
        {overlap: "worst", version: "A", RF.iteration: "i+1", RF.start: 2, RF.end: 3},
        {overlap: "worst", version: "A", RF.iteration: "i+2", RF.start: 4, RF.end: 5},
        {overlap: "worst", version: "B", RF.iteration: "j", RF.start: 1, RF.end: 2},
        {overlap: "worst", version: "B", RF.iteration: "j+1", RF.start: 3, RF.end: 4},
        {overlap: "worst", version: "B", RF.iteration: "j+2", RF.start: 5, RF.end: 6},
    ]
)

df[RF.start] = now + df[RF.start].apply(lambda x: timedelta(minutes=x))
df[RF.end] = now + df[RF.end].apply(lambda x: timedelta(minutes=x))
fig = px.timeline(
    df, x_start=RF.start, x_end=RF.end, y=version, text=RF.iteration, facet_col=overlap
)
fig.update_layout(height=300)
fig.update_yaxes(categoryorder="category descending")
fig.write_image(f"{FIGURES}/{name}.pdf")
fig.show()

In [ ]:
name = "overlap_interference"

now = datetime(year=2022, month=7, day=1, hour=8, minute=20, second=0)
type = "Type"
cpu = "CPU bound"
io = "I/O bound"

df = pd.DataFrame(
    [
        {version: "A", RF.start: 0, RF.end: 5, type: cpu},
        {version: "B", RF.start: 0, RF.end: 6, type: cpu},
        {version: "A", RF.start: 5, RF.end: 10, type: io},
        {version: "B", RF.start: 6, RF.end: 12, type: io},
        {version: "A", RF.start: 11, RF.end: 16, type: cpu},
        {version: "B", RF.start: 13, RF.end: 19, type: cpu},
        {version: "A", RF.start: 16, RF.end: 21, type: io},
        {version: "B", RF.start: 19, RF.end: 24, type: io},
        {version: "A", RF.start: 22, RF.end: 27, type: cpu},
        {version: "B", RF.start: 25, RF.end: 30, type: cpu},
        {version: "A", RF.start: 27, RF.end: 32, type: io},
        {version: "B", RF.start: 30, RF.end: 35, type: io},
    ]
)

cpu_time = 4
io_time = 4
wait_time = 1

b_rigs = 2
b_rig_time = 1

a_time = 0
b_time = 0
data = []
for repetition in range(5):
    data.append({version: "A", RF.start: a_time, RF.end: a_time + cpu_time, type: cpu})
    a_time += cpu_time
    data.append({version: "A", RF.start: a_time, RF.end: a_time + io_time, type: io})
    a_time += io_time + wait_time

    b_rig = 0
    if repetition < b_rigs:
        b_rig = b_rig_time
        b_rig_time *= 0.9
    data.append(
        {version: "B", RF.start: b_time, RF.end: b_time + cpu_time + b_rig, type: cpu}
    )
    b_time += cpu_time + b_rig
    data.append(
        {version: "B", RF.start: b_time, RF.end: b_time + io_time + b_rig, type: io}
    )
    b_time += io_time + b_rig + wait_time


df = pd.DataFrame(data)
df[RF.start] = now + df[RF.start].apply(lambda x: timedelta(minutes=x))
df[RF.end] = now + df[RF.end].apply(lambda x: timedelta(minutes=x))
fig = px.timeline(
    df,
    x_start=RF.start,
    x_end=RF.end,
    y=version,
    color=type,
    color_discrete_sequence=px.colors.qualitative.Plotly[2:4],
)
fig.update_layout(height=300)
fig.update_yaxes(categoryorder="category descending")
fig.write_image(f"{FIGURES}/{name}.pdf")
fig.show()